In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
import psrc_theme

#to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [3]:
# get shared data
%store -r validation_data

#config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

per_data = validation_data.persons_data.copy()

# delete shared data
del validation_data



In [4]:
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_des'] = per_data['ptype'].map(ptype_cat)

telecommute_cat = {"No_Telecommute": "0 day",
                   "1_day_week": "1 day",
                   "2_3_days_week": "2-3 days",
                   "4_days_week": "4 days"}
per_data['telecommute_frequency_des'] = per_data['telecommute_frequency'].map(telecommute_cat)

wfh_cat = {True: "wfh worker",
           False: "on-site worker"}
per_data['work_from_home_des'] = per_data['work_from_home'].map(wfh_cat)

## Day activity pattern distribution

- **Mandatory**: the person engages in travel to at least one out-of-home mandatory activity - work, university, or school. The mandatory pattern may also include non-mandatory activities such as separate home-based tours or intermediate stops on mandatory tours.
- **Non-mandatory**: the person engages in only maintenance and discretionary tours, which, by definition, do not contain mandatory activities.
- **Home**: the person does not travel outside the home.

In [4]:
df_plot = per_data.groupby(['source','cdap_activity'])['person_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight'].\
            apply(lambda x: x / float(x.sum()))

# df_plot
fig = px.bar(df_plot, x="cdap_activity", y="percentage", barmode="group",color="source",
            title="CDAP")
# fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"))
fig.show()

## CDAP activity by segment

In [5]:
df_plot = per_data.groupby(['source','ptype_des','cdap_activity'])['person_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','ptype_des'], group_keys=False)['person_weight'].\
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="cdap_activity", y="percentage", color="source",barmode="group",
                facet_col="ptype_des", facet_col_wrap=2,
                title="CDAP by person types")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
fig.update_layout(height=900, width=800)
fig.show()

In [6]:
df_plot = per_data.groupby(['source','telecommute_frequency_des','cdap_activity'])['person_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','telecommute_frequency_des'], group_keys=False)['person_weight'].\
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="cdap_activity", y="percentage", color="source",barmode="group",
                facet_col="telecommute_frequency_des", facet_col_wrap=2,
                title="CDAP by telecommute frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
fig.update_layout(height=500, width=800)
fig.show()

In [7]:

df_plot = per_data.groupby(['source','work_from_home_des','cdap_activity'])['person_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source','work_from_home_des'], group_keys=False)['person_weight'].\
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="cdap_activity", y="percentage", color="source",barmode="group",
                facet_col="work_from_home_des", facet_col_wrap=2,
                title="CDAP by work from home worker")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
fig.update_layout(height=400, width=800)
fig.show()